# Scrapy

És un framework Python que serveix per fer web scraping. Conté les eines necessàries per facilitar-nos la tasca d'obtenir dades de webs, processar-les i guardar-les.

El nostre objectiu en aquesta xerrada serà escrapejar l'agenda de la PyConES que es farà al setembre.

## Procés

* Trobar la URL que ens retorna la informació que volem.
* Identificar la informació que volem aconseguir.
* Trobar els XPath.



La url on hi ha la informació és aquesta: https://2017.es.pycon.org/ca/schedule/

Ara cal tenir pràctica amb la consola del navegador per entendre l'HTML i muntar els Xpaths que necessitem.
Hi ha un plugin del Chrome que és molt útil per comprovar els Xpath, es diu: Xpath Helper: https://chrome.google.com/webstore/detail/xpath-helper/hgimnogjllphhhkhlmebbmlgjoejdpjl

També n'hi ha un per Firefox, Xpath Checker: https://addons.mozilla.org/ca/firefox/addon/xpath-checker/

Amb aquests plugins podem escriure els xpaths i veurem què és el que estem recollint al moment.

El procediment és buscar la informació que volem a l'HTML, veure on hi ha bucles, etc i després fer els Xpath.

### Què és un XPath?
És un llenguatge que permet navegar per etiquetes mitjançant els tipus d'etiquetes i els seus atributs. Es pot utilitzar en XML o HTML. I es pot obtenir qualsevol dada que estigui dins el codi HTML.
És important fer XPath **reutilitzables i sobretot genèrics i entendors**. Ja que es poden fer Xpath comptant divs, etc i no és una bona pràctica perquè és molt complicat seguir el fil per una altre persona o per un mateix. També és molt important que siguin genèrics i que utilitzin atributs únics o representatius com ids. Estem accedint a un codi que no és nostre i que és possible que canviï, així doncs, **si fem els Xpath genèrics i robustos no haurem de canviar-los al mínim canvi de la web**.

### Què és una petició i resposta HTTP?
HTTP és el protocol que fa servir la web per comunicar-se entre els clients i els servidors.
Un client fa una petició HTTP a una web perquè vol veure la web esperant que el servidor li retorni una resposta.
En el nostre cas, Scrapy farà peticions a les webs que nosaltres li indiquem i rebrà la resposta on hi haurà tot el codi HTML de la web que després podrem tractar.

## Iniciar el projecte
Per iniciar un projecte de scrapy hem d'executar això:

In [18]:
scrapy startproject pycon

Això ens crearà una estructura de carpetes:

Com podem veure tenim:
* Items
* Middlewares
* Pipelines
* Settings
* Spiders

### Items
Són classes que s'utilitzen per organitzar les dades que volem guardar. És opcional, però és recomenable ja que permet que es desin les dades automàticament en JSON, etc.
També ajuda a no fer errors de sintaxis en la sortida. 

### Middlewares
És una classe amb certs mètodes que serveixen per manipular les peticions i respostes.

### Pipelines
Són classes que s'executen quan es disparen certs events. Com per exemple, a l'inici d'un spider o al final.
S'utilitzen per manipular les dades obtingudes del spider i per guardar-les en fitxers, bases de dades, etc.

### Settings
És l'arxiu on hi ha la configuració, per exemple aquí és on es defineixen les classes dels pipelines que volem que s'activin, l'ordre, etc.

### Spiders
Són les classes que cal programar perquè accedeixin a la web, recuperin el que ens interessi i es guardi.

#### Funcionament dels spiders
1. Tenen un atribut **name** amb el qual scrapy l'identifica, ha de ser únic, no pot haver-hi dos spiders amb el mateix nom.
2. Tenen un atribut **start_urls** que és una llista de les urls a les quals accedirem.
3. Tenen varis mètodes:
    1. **start_requests**(opcional): serveix per modificar la petició i enviar-la amb el que s'hagi volgut canviar.
    2. **parse**: rep per paràmetre la resposta de la petició, aquí és on hi haurà la major part de la lògica. És on hi hauran els Xpath i on obtindrem la informació que volem.

## Com funciona Scrapy?
1. Hem de tenir creat els nostres spiders.
2. Quan executem scrapy indiquem quin spider es vol executar, s'indica amb el seu **name**.
3. El primer que s'executa del spider és el constructor.
4. Després s'executa el mètode **start_requests** (si hi és). Si hi és generarà el Request i enviarà el Response al callback indicat. Sinó, scrapy fa la Request.
5. Si tenim un middleware de Request l'interceptarà abans de fer la petició.
6. Fa la petició.
7. Si tenim un middleware de Response l'interceptarà.
8. S'executa el mètode **parse** o el mètode del callback que hagi definit **start_requests** (si s'ha definit).
9. Si tenim un pipelines definit s'executarà cada vegada que el mètode **parse** retorni un item. I finalment quan el spider es tanqui.

## Executar un spider

## Evitar baneig de IP
Hi ha varies maneres de fer-ho, però totes tenen el mateix objectiu, no fer peticions amb la IP real. Per això s'utilitzen proxies.
Una manera més avançada que utilitzar un llistat estàtic de proxies és utilitzar **Tor**

### Què és Tor?
És una xarxa mundial on t'hi connectes i et genera un circuit de 5 ordinadors. Això significa que et connectes a un ordinador, aquest es connecta a un segon i així fins a 5, per tant, cada petició que facis passarà per aquest circuit i és molt complicat de restrejar qui està al final del circuit.
Els circuits s'actualitzen, etc. Es pot configurar de moltes maneres però aquesta és la per defecte.

#### Tor funciona amb SOCKS, Scrapy no ho suporta
Tor funciona amb el protocol SOCKS i Scrapy no ho permet, només funciona amb HTTP, per tant necessitem un traductor que ens converteixi una connexió SOCKS amb una HTTP, això ho fa **Polipo** o **Privoxy**.

### Instal·lació de Tor i Polipo

In [ ]:
sudo apt-get install tor polipo
sudo service tor stop
sudo service polipo stop

### Iniciar Tor i Polipo

In [ ]:
# Tor per defecte obre el port 9050
tor &
# Si volem especificar un port
mkdir data
tor --SocksPort 9051 --DataDirectory data/tor2 &
# Iniciem Polipo indicant-li el port de Tor i assignem el port de Polipo
sudo polipo socksParentProxy=localhost:9050 proxyPort=14000 &
sudo polipo socksParentProxy=localhost:9051 proxyPort=14001 &

### Comprovació d'IP

In [ ]:
curl "http://cualesmiip.es" --proxy localhost:14000
curl "http://cualesmiip.es" --proxy localhost:14001

### Integrar Tor i Polipo a Scrapy
El que hem de fer és que totes les peticions que es facin passin pel port de Polipo que tenim obert. Això ho farem manipulant els headers de la petició HTTP.
Per a fer això creem un middleware que intercepti totes les Requests que es llencin i hi afegim un header proxy en el meta d'aquesta manera:

In [ ]:
class ProxyDownloaderMiddleware(object):
    def process_request(self, request, spider):
        request.meta['proxy'] = 'localhost:14000'

### Modificar User-Agent
És molt important canviar el User-Agent de les peticions que fem a través de Scrapy ja que sinó és molt fàcil veure que és un robot i no una persona qui fa les peticions. Hi ha varies maneres de fer-ho:
* Utilitzar la configuració de Scrapy que ens posa algun tipus de User-Agent estàtic.
* Tenir una llista de User-Agents i muntar una funció que ens en retorni un aleatoriament i posar-lo dins les peticions. Cal un manteniment per actualitzar els user-agents.
* Utilitzar fake_user_agent, és un paquet que está a Pypi i que va actualitzant la DB de user-agents. Per mi és la millor opció. Tot i que és una llibreria més, tot depèn del projecte que volem fer, si és un projecte gran jo ho utilitzaria.

#### Com posar User_Agent a les peticions
Amb un middleware igual que el proxy, podem modificar totes les peticions afegint-hi el header del user-agent.
Hi ha algun middleware per defecte de Scrapy de User-Agent.